In [ ]:
!pip install "pip<24.1"
!pip install textract


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 0s (734 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!apt-get install -y tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (14.0 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124977 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [ ]:
import os
import spacy
import re
from dateutil import parser
import textract
import json


In [ ]:
nlp = spacy.load("en_core_web_sm")


In [ ]:
import os

# Folder containing the PDF files
folder_path = r'/content/'

# Check if the folder exists
if os.path.exists(folder_path):
    # Get all PDF files in the folder
    all_pdf = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.pdf')]

    # Print the list of PDF file paths
    print("List of PDF files:")
    for pdf in all_pdf:
        print(pdf)
else:
    print(f"Folder does not exist: {folder_path}")





List of PDF files:
/content/INV_UCB_19847612.pdf
/content/201051107425035.pdf
/content/202076540912.pdf
/content/ISMP23497211.pdf
/content/09f8cfb0-d649-0137.pdf


In [ ]:
import os
print(os.path.exists(r'C:\Users\VIBHOR\Desktop\Sample_Files'))

all_pdf = [r'C:\Users\VIBHOR\Desktop\Sample_Files\09f8cfb0-d649-0137.pdf',
           r'C:\Users\VIBHOR\Desktop\Sample_Files\201051107425035.pdf']


False


In [ ]:
def Is_Invoice_Or_something_else(text):

    tokens = text.lower().split()

    valid_invoice_tags = ['invoice', 'lading', 'bill']

    output = [i for i in tokens if i in valid_invoice_tags]

    if len(output) > 0:
        return "Invoice"
    else:
        return "Wrong Invoice Files"

In [ ]:
def Clean_Text(file):
  text = textract.process(file,method='tesseract')

  #convert into string
  text = text.decode('utf-8')
  #replace "\r\n" with space
  text = text.replace("\r\n"," ")
  #remove double space
  text = re.sub(" +"," ", text)

  return text

In [ ]:
def Get_Tag_Position(tag, text, direction = 'forward'):

    tag = tag.lower()

    if direction == 'reverse':
        tag_pos = text.lower().rfind(tag)
    else:
        tag_pos = text.lower().find(tag)

    if tag_pos > 0:
        start_pos = tag_pos + len(tag) + 1
    else:
        start_pos = -1

    return start_pos

In [ ]:
def Get_Invoice_Date(text):

    possible_dates = []

    # Search for the exact tag mentioned as "Invoice Date"
    temp_corpus = text[Get_Tag_Position("invoice date", text):]
    doc = nlp(temp_corpus)
    possible_dates = [ent for ent in doc.ents if ent.label_ == "DATE"]


    # If the tag "Invoice Date" is not found then search for the tag "Date"
    temp_corpus = text[Get_Tag_Position("date", text):]
    doc = nlp(temp_corpus)
    possible_dates += [ent for ent in doc.ents if ent.label_ == "DATE"]


    # If both the above fail then loop through all the tokens and try to identify valid dates
    tokens = text.split()

    for token in tokens:

        if len(token) >= 4:
            try:
                converted_date = parse(token)
                possible_dates.append(token)
            except:
                continue

    # Convert all the date tokens found to string
    possible_dates = list(map(str, possible_dates))


    # Select only the final valid dates
    valid_dates = []

    for date in possible_dates:

        if len(date) >= 4:
            try:
                converted_date = parse(date)
                valid_dates.append(date)
            except:
                continue

    if len(valid_dates) > 0:
        return valid_dates[0]
    else:
        return "No Invoice Date Found"

In [ ]:
#Get invoice Amount
def Parse_Amounts(text):
  money = re.compile('|'.join([
       r"\$?\d+\.?\d+\,\d{1,2} ",  # EUR format
       r"\$?\d+\,?\d+\.\d{1,2} ",  # USD format
       r"\$\d+\,?\d*\.?\d{1,2} ",  # USD format without decimals

  ]))
  matches = re.findall(money,text)
  matches = [i.strip() for i in matches]

  return matches



In [ ]:
def Get_Invoice_Amount(text):

    text = text.lower()

    invoice_amount = ""

    amounts = Parse_Amounts(text)

    if len(amounts) > 0:
        invoice_amount = amounts[-1]
    else:

        temp_corpus = text[Get_Tag_Position("total", text, direction='reverse'):]
        numbers = re.findall("\d+", temp_corpus)

        if len(numbers) > 0:
            invoice_amount = numbers[0]

    return invoice_amount



In [ ]:
#Get Invoice Number

def Extract_Tag_Data(tag, text, direction = 'forward'):

    token = ''

    start_pos = Get_Tag_Position(tag, text, direction=direction)

    if start_pos > 0:
        temp_corpus = text[start_pos:]
        token = temp_corpus.split('|')[0]

    return token

In [ ]:
def Get_Invoice_Number(file):

    # Preprocess the text. Using pipe delimited format as the invoice number has spaces in some cases
    #------------------------------------------------------------------------------------------------

    text = textract.process(file, method='tesseract')
    text = text
    text = text.replace("\r\n", "|")
    text = re.sub(" +", " ", text)

    tokens = [i for i in text.split('|') if len(i) > 1]
    text = '|'.join(tokens)

    text = text.lower()
    text = re.sub(" number:| no\.| n\.", " number", text)


    # Extract the Invoice Number
    #---------------------------
    valid_matches = []

    keywords = ["invoice number", "lading number", "invoice", "lading", "number"]

    for keyword in keywords:

        token = Extract_Tag_Data(keyword, text, 'reverse')

        if len(token) > 0:
            valid_matches.append(token)

    valid_matches = [i.upper() for i in valid_matches]

    if len(valid_matches) > 0:
        invoice_number = valid_matches[0]
        if len(invoice_number) > 20:
            invoice_number = invoice_number.split('/')[0]
    else:
        invoice_number = "No Invoice Number found"

    return invoice_number

In [ ]:
#Packaging all together
for file in all_pdf:

    print("Processing File :", file)

    file_text = Clean_Text(file)

    if Is_Invoice_Or_something_else(file_text) == "Invoice":

        Invoice_Number = Get_Invoice_Number(file)
        Invoice_Date = Get_Invoice_Date(file_text)
        Invoice_Amount = Get_Invoice_Amount(file_text)

        print("Document Type : Invoice")
        print("Invoice Number :", Invoice_Number)
        print("Invoice Date :", Invoice_Date)
        print("Invoice Amount :", Invoice_Amount)
        print("\n")

    else:

        print("you got the wrong bill")


Processing File : /content/INV_UCB_19847612.pdf
Document Type : Invoice
Invoice Number : 90000178
DOCUMENT DATE: 08 JANUARY 2019

 

 

PACKAGING INFORMATION
NUMBER OF PIECES
2 PALLET

TOTAL NUMBER OF UNITS: 2 PIECE

 

INVOICE DETAILS

VAT REGISTRATION NO: BE0810940487

PAYER: 20000391

YOUR REFERENCE: BEBRA00003420904

OUR REFERENCE: 93209634

ORDER DATE: 21 SEP 2018

TERMS OF DELIVERY: EXW BRAINE L’ALLEUD
MODE OF TRANSPORT: BOAT - COLLECTED

CURRENCY: EUR

 

 

MATERIAL CODE DESCRIPTION BATCH QUANTITY AMOUNT
VALUE
CIA03203 KEPPRA IV SOOMG
Invoice Date : No Invoice Date Found
Invoice Amount : 31.989,48


Processing File : /content/201051107425035.pdf
Document Type : Invoice
Invoice Number : TERRELL RICHEMOND VIVAMUS IN FELIS EU SAPIEN CURSUS

VESTIBULUM. PROIN EU MI. NULLA AC ENIM.

IN TEMPOR, TURPIS NEC EUISMOD

YRUAN@EXAMPLE.COM SCELERISQUE, QUAM TURPIS ADIPISCING
LOREM, VITAE MATTIS NIBH LIGULA NEC SEM.

EMAIL

PHONE NUMBER

+33 (13) 634-6159 INVOICE DATE
NOVEMBER 6, 1971 22:28

